# **LAB 1: DATA PREPARATION**

In this notebook, we will explore how do prepare data for downstream NLP tasks.  

A non-exhaustive list tasks related to NLP and Generative AI that can benefit from this data
preparation is:

- Question & Answer
- Text Summarization
- Instruct Tuning
- Human-Bot Conversations
- Continued PreTraining


For each of these tasks, there are subtle differences in how one might want to prepare the data.
Some of the key steps one might use here for data prep are also applicable to 
applying guardrails to LLMs (e.g., `Profanity Check` and `Toxicity Detection`)

More on guardrails in Lab #5, but for now let's import our required python 
libraries and get started!


### Step 1: Persistent Python environment

[Implementation](https://www.kaggle.com/code/kononenko/pip-install-once)
[pip-install-forever](https://www.kaggle.com/code/samsammurphy/pip-install-forever)

- Adjust the session persistence settings. 
  - Go to Notebook options (right pane) and set PERSISTENCE to Files only. You can also set it to Variables and Files, if you need persistent variables for your own reasons.
  - Install in target directory: `pip install -r requirements.txt --target=/kaggle/working/workshop`
  - Next:
    1. Import sys: `import sys`
    2. Add path: `sys.path.append("/kaggle/working/workshop")`

In [ ]:
!wget 

In [1]:
!pip install -r requirements_lab1.txt

     |████████████████████████████████| 130 kB 992 kB/s eta 0:00:01
     |████████████████████████████████| 260 kB 870 kB/s eta 0:00:01
     |████████████████████████████████| 73 kB 910 kB/s eta 0:00:011
     |████████████████████████████████| 1.0 MB 1.1 MB/s eta 0:00:01     |███████████████▎                | 491 kB 400 kB/s eta 0:00:02
     |████████████████████████████████| 226 kB 471 kB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 819 kB/s eta 0:00:01
     |████████████████████████████████| 647 kB 526 kB/s eta 0:00:01
  Using cached anyio-4.0.0-py3-none-any.whl (83 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached blessed-1.20.0-py2.py3-none-any.whl (58 kB)
  Using cached botocore-1.31.64-py3-none-any.whl (11.3 MB)
     |████████████████████████████████| 158 kB 514 kB/s eta 0:00:01
     |████████████████████████████████| 13

In [2]:
!pip install profanity_check --target=/kaggle/working/workshop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 29.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.1 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, 

In [3]:
import sys
sys.path.append("/kaggle/working/workshop")

# Import Required Dependencies 

In [4]:
import warnings
warnings.filterwarnings('ignore')

# set flag for training environment
TRAINING = True

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import numpy as np
from profanity_check import predict_prob

import matplotlib

ImportError: cannot import name 'joblib' from 'sklearn.externals' (/opt/conda/lib/python3.10/site-packages/sklearn/externals/__init__.py)

# Dataset

Training dataset: Text content and metadata for LinkedIn posts during 2021. These posts were collected from the
internet and correspond to a wide array of identified "influencers".

Source: https://www.kaggle.com/datasets/shreyasajal/linkedin-influencers-data


# Import Dataset

To the right of this page, go to "Add Data" and search for "LinkedIn Influencers' Data". Hit the "+" icon to add the data to your `/kaggle/input` data directory.

In [ ]:
# Read in from public S3 Location
df = pd.read_csv("/kaggle/input/linkedin-influencers-data/influencers_data.csv")
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

There are approximately ~70 LinkedIn influencers in this dataset. Let's take a look at
how many posts we have data on for each influencer.

In [ ]:
# count how many records we have for influencers
df.name.value_counts().sort_values().plot(kind='barh', figsize=(8,12))

# Create Small Sample DataFrame

In [ ]:
sample_df = df.sample(100)[['name', 'headline', 'content']]

# drop records with missing content
sample_df = sample_df.dropna()

sample_df.head()

# Custom Functions

Set up a custom function to help with the data transformation. 

```
def custom_function(df, text_columns):
    
    # For text cleaning based functions use: 
    for column in text_columns: 
        df[column] = # add change code here 
    
    #For an example of how to filter the df see number_proportion_filter.py or numeric_filer.py 
    
    return df
```


# 🙈 Profanity Check




In [ ]:
from profanity_check import predict_prob

threshold = 0.9

sample_df["profanity"] = predict_prob(sample_df["content"])

sample_df = sample_df[sample_df["profanity"] < threshold]
sample_df = sample_df.reset_index(drop=True)
sample_df.head()


In [ ]:
sample_df['profanity'].hist()

# 🧐 Text Quality Check

## Flesch Grade reading level

Depending on the task at hand, sometimes it's advantages to evaluate text and
language based on it's estimated reading level. One popular way to do this 
is with the Flesch-Kincaid grade level.  This is a grade formula in that a score
of 9.3 means that a ninth grader would be able to read the document.

The wikipedia article on this technique does a fair job explaining it:


Using the `textstat` python linvrary, we can approximate the reading level and
set our thresholds for what level we find ideal for our downstream NLP task.

In this case, we will limit the reading level between `2` and `10`, but many
other ranges are certainly possible

In [ ]:
import textstat 

min_grade_level = 2
max_grade_level = 10

def check_quality(df, text_column):
    df["flesch_grade"] = df[text_column].apply(textstat.flesch_kincaid_grade)
    return df

cleaned = check_quality(sample_df, 'content')

# Filter to only those records with content between 2 and 10 grade level
cleaned = cleaned[(cleaned["flesch_grade"] >= min_grade_level) & (cleaned["flesch_grade"] <= max_grade_level)]
cleaned.head()


In [ ]:
sample_df['flesch_grade'].hist()

# Run Preparation Pipeline

In [ ]:
# Basic ETL

# Read in some additional utility functions

def remove_trailing_ws(df, text_columns):
    # For text cleaning based functions use: 
    for column in text_columns: 
        df[column] = df[column].str.strip()
    return df

def length_check_func(df,text_column, minLength,maxLength):
    df = df[(df[text_column].str.len() > int(minLength))]
    df = df[(df[text_column].str.len() < int(maxLength))]
    return df

def string_replace(df, text_columns, input, output):
    # For text cleaning based functions use: 
    for column in text_columns: 
        df[column] = df[column].str.replace(input, output)
    return df

# Keep Relevant Columns
cleaned = df[['name', 'headline', 'about', 'content', 'reactions']]

# Dropping missing
cleaned = cleaned.dropna()


# Text Cleaning (Minimal)
cleaned = string_replace(cleaned, ['content'], '…see more', '')
cleaned = remove_trailing_ws(cleaned, ['content'])

max_len = 10000
min_len = 1

cleaned = length_check_func(cleaned, 'content', min_len, max_len)


# Set thresholds

# Profanity Threshold
profanity_threshold = 0.9

# Target Reading Level
min_grade_level = 2
max_grade_level = 10

# Predict Probability of Profane Language
cleaned["profanity"] = predict_prob(cleaned["content"])

# Filter out Profane Language
cleaned = cleaned[cleaned["profanity"] < profanity_threshold]
cleaned = cleaned.reset_index(drop=True)

# Determine Reading Level
cleaned = check_quality(cleaned, 'content')

# Filter to only those records with content between a 2nd and 10th grader
cleaned = cleaned[(cleaned["flesch_grade"] >= min_grade_level) & (cleaned["flesch_grade"] <= max_grade_level)]
cleaned.head()


In [ ]:
num_dropped = df.shape[0] - cleaned.shape[0]

print(f"Lost a total of {num_dropped} records, or about {round(num_dropped/df.shape[0], 2) * 100}%")

# One Step Further

If we really want to talk like an influencer, perhaps we should also additionally limit the number of records
by the number of reactions the posts got. Let's take a look.

In [ ]:
cleaned.sort_values('reactions')

We can see that the number of reactions these posts from 'influencers' received appear to go 
from zero (yikes!) all the way to over 330k. Noticeably, that single post with ~330k reactions
is simply `Helen is my kinda lady`.

The fact this post received so much attention given it's lack of context is also 
a warning that there could be exogenous latent variables, such as current events, pop culture, etc, that could be driving 
the number of reactions, not necessarily the content itself. 

Let's be just a little more analytical:

In [ ]:
# Identify the 90th percentile of reactions over all posts
p90 = np.quantile(cleaned.reactions, 0.9)

print(f'The 90th percentile is {p90} reactions.')


For our purposes, let's focus on the top "performing" content to fine tune our model on

In [ ]:
cleaned = cleaned[cleaned.reactions > p90]
cleaned.shape

We're now left with 2,127 high-quality data points to experiment with 
fine tuning on. 

Finally, let's ask h2oGPT to provide a title for our LinkedIn Influencer content.
This process is called `zero-shot text generation` (more on this later)

In [ ]:
sample_df = cleaned.sample(5)

from gradio_client import Client
import ast
from pprint import pprint


HOST_URL = "https://gpt-genai.h2o.ai/"
GPT_KEY = "f74f043e-45fc-4dfe-9c33-55a4720427f6"
    
client = Client(HOST_URL)

from tqdm import tqdm
tqdm.pandas()

def generate_title(content):
    
    #try:
    summarize_prompt = 'You are a helpful, respectful and honest assistant the specializes in generating accurate titles of LinkedIn posts. Provide a title for the following post. The title should be a single sentence, not using bullet points. Only include the title in the response. The LinkedIn post is: ' + content
    kwargs = dict(
        instruction_nochat=summarize_prompt, 
        h2ogpt_key = GPT_KEY)
    
    response = client.predict(str(dict(kwargs)), api_name='/submit_nochat_api')
    reply = ast.literal_eval(response)['response']
    #except:
        #reply = 'NA'
    return reply
        
sample_df['title'] = sample_df.progress_apply(lambda row :   generate_title(row['content']), axis=1)
sample_df.head()

In [ ]:
pprint(f'CONTENT:')
sample_df['content'].to_list()

In [ ]:
print(f'TITLE:')
sample_df['title'].to_list()

In [ ]:
# Optional alternative for fine tuning - create an instruction.
cleaned['instruction'] = ('Write a LinkedIn post in the style of an influencer whom has the title of '
  + cleaned['headline'] + ' and can be described by the following: ' 
  + cleaned['about'])

cleaned.sample(5)

# Output Dataset

Now we're ready to store our data set out and experiment with fine-tuning
in Lab # 2 with H2O LLM Studio. 

In [ ]:
#######################################################################################
# WARNING! This could take a very long time using the public facing h2oGPT endpoint. ##
#######################################################################################

# Apply to full data set
cleaned['title'] = cleaned.progress_apply(lambda row :   generate_title(row['content']), axis=1)

In [ ]:
# Output locally
cleaned.to_csv('/kaggle/working/influencers_data_prepared.csv', index=False)

---

# Understanding Data Prep for Production

In this lab, we cleaned the data and fine-tuned with with Title column added from the Content field. 

Other pipeline components include:

- Data Augmentation
- Text Cleaning
- Profanity Check 
- Text Quality Check
- Length Checker 
- Valid Question
- Pad Sequence
- Truncate Sequence by Score 
- Compression Ratio filter
- Boundary Marking
- Sensitive Info Checker
- RLHF Protection
- Language Understanding 
- Data Deduplication
- Toxicity Detection